# Вебинар 4. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
        
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items

In [2]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [5]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, item_features=item_features, take_n_popular=5000, category=['',])

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

/home/mrb/Rec_sys_GB/utils.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 86865 to 5001


In [6]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,818980,818981,819063,819255,819304,819308,819330,819518,819594,819765,...,15596515,15629778,15630096,15741861,15778533,15831255,15926712,15926775,15926844,15926886
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [8]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [9]:
%%time

model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 27.7 s, sys: 41.3 s, total: 1min 9s
Wall time: 8.97 s


-----

# Домашнее задание

1. Перенесите метрики в модуль metrics.py (убедится что они там)
2. Перенесите функцию prefilter_items в модуль utils.py
3. Создайте модуль recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в recommenders.py
4. Проверьте, что все модули корректно импортируются

In [10]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS

    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """

    def __init__(self, data, N=5, weighting=True):
        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать

        self.data = data
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
            self.itemid_to_id, self.userid_to_id = self.prepare_dicts()
        self.popularity = self.get_n_popularity_item(N)

        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T

        self.model = self.fit()
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

    def prepare_matrix(self, data):
        """Создаем user-item матрицу"""

        user_item_matrix = pd.pivot_table(data,
                                          index='user_id', columns='item_id',
                                          values='quantity',  # Можно пробовать другие варианты
                                          aggfunc='count',
                                          fill_value=0
                                          )

        user_item_matrix = user_item_matrix.astype(float)  # необходимый тип матрицы для implicit

        return user_item_matrix

    def prepare_dicts(self):
        """Подготавливает вспомогательные словари"""

        self.userids = self.user_item_matrix.index.values
        self.itemids = self.user_item_matrix.columns.values

        self.matrix_userids = np.arange(len(self.userids))
        self.matrix_itemids = np.arange(len(self.itemids))

        self.id_to_itemid = dict(zip(self.matrix_itemids, self.itemids))
        self.id_to_userid = dict(zip(self.matrix_userids, self.userids))

        self.itemid_to_id = dict(zip(self.itemids, self.matrix_itemids))
        self.userid_to_id = dict(zip(self.userids, self.matrix_userids))

        return self.id_to_itemid, self.id_to_userid, self.itemid_to_id, self.userid_to_id

    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""

        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())

        return own_recommender

    def fit(self, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""

        model = AlternatingLeastSquares(factors=n_factors,
                                        regularization=regularization,
                                        iterations=iterations,
                                        num_threads=num_threads)
        model.fit(csr_matrix(self.user_item_matrix).T.tocsr())

        return model

    def get_n_popularity_item(self, N=5):
        """Берем топ-N товаров юзеров"""

        popularity = self.data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        popularity.sort_values('quantity', ascending=False, inplace=True)

        popularity = popularity[popularity['item_id'] != 999999]

        popularity = popularity.groupby('user_id').head(N)

        popularity.sort_values(by=['user_id', 'quantity'], ascending=False, inplace=True)

        return popularity

    def get_rec_similar_items(self, x):
        """Берем товар наиболее похожий на целевой"""

        recs = self.model.similar_items(self.itemid_to_id[x], N=2)
        top_rec = recs[1][0]

        return self.id_to_itemid[top_rec]

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        popularity = self.popularity.loc[self.popularity['user_id'] == user]
        popularity['similar_recommendation_bpr'] = popularity['item_id'].apply(lambda x: self.get_rec_similar_items(x))

        recommendation_similar_items = popularity.groupby('user_id')['similar_recommendation_bpr']. \
        unique().reset_index()
        recommendation_similar_items.columns = ['user_id', 'similar_recommendation_bpr']
        res = recommendation_similar_items['similar_recommendation_bpr'].loc[
            recommendation_similar_items['user_id'] == 1].tolist()[0]
        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res


    def get_rec_similar_users(self, x, N=16, treshold=0.45):
        """Получаем список юзеров, похожих на целевого юзера"""

        simil_usr = self.model.similar_users(self.userid_to_id[x], N)[1:]
        simil_usr_list = np.array([list(x) for x in simil_usr if x[1] > treshold])[:, 0].astype('int').tolist()

        return [self.id_to_userid[x] for x in simil_usr_list]


    def get_list_items_simil_usrs(self, x, N=16, treshold=0.45):
        """Получаем и сортируем по попуярности товары, купленные похожими юзерами"""

        simil_usr_list = self.get_rec_similar_users(x, N, treshold)
        list_items = self.data[self.data['user_id'].isin(simil_usr_list)]
        list_items = list_items.groupby(['item_id'])['quantity'].count().reset_index()
        list_items.sort_values('quantity', ascending=False, inplace=True)

        return list_items


    def get_similar_users_recommendation(self, user, N=5, n_users=16, treshold=0.45):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""

        popularity_items_users = self.get_list_items_simil_usrs(user, n_users, treshold)
        res = popularity_items_users['item_id'].head(N).values

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res


----

Проверка, что все работает

In [11]:
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender



In [13]:
recommend = MainRecommender(data_train, weighting=True)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [29]:
result = recommend.filter_result(data_test)

In [30]:
result

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
...,...,...
2037,2496,[6534178]
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ..."
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972..."
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813..."


In [31]:
%%time
result['similar_items'] = result['user_id'].apply(lambda x: recommend.get_similar_items_recommendation(x))

print("Precision:",result.apply(lambda row: precision_at_k(row['similar_items'], row['actual']), axis=1).mean())

/home/mrb/Rec_sys_GB/recommenders.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popularity['similar_recommendation_bpr'] = popularity['item_id'].apply(lambda x: self.get_rec_similar_items(x))


Precision: 0.08234688625836335
CPU times: user 23.8 s, sys: 1min 10s, total: 1min 34s
Wall time: 12.3 s


In [33]:
%%time
result['similar_items'] = result['user_id'].apply(lambda x: recommend.get_similar_users_recommendation(x))

print("Precision:",result.apply(lambda row: precision_at_k(row['similar_items'], row['actual']), axis=1).mean())

Precision: 0.09933093154915079
CPU times: user 2min 29s, sys: 7min 8s, total: 9min 37s
Wall time: 1min 15s


Замучался с ассертами, но допилил фильтрацию result, чтобы туда не попадали юзеры с меньшим количеством рекомендаций.
Оказывается рекомендации по похожим юзерам, судя по всему более эффективнее, чем по похожим товарам из личного топа.